In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-256399
Azure region: southcentralus
Subscription id: cdbe0b43-92a0-4715-838a-f2648cc7ad21
Resource group: aml-quickstarts-256399


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "cluster-project1"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

from azureml.core.compute_target import ComputeTargetException

try:
    compute_target = ComputeTarget(workspace = ws, name = cluster_name)
    print('found existing cluster, use it')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size = 'Standard_D2_V2', max_nodes = 4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output = True)   

found existing cluster, use it

Running


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C': choice(0.1, 1, 10, 100),
        '--max_iter': choice(100, 200, 300, 500)
    }
)
# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='./',
                      script='train.py',
                      arguments=[],
                      compute_target=compute_target,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src, 
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name='Accuracy', #we need to log this metric in train.py
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                             max_total_runs=4,
                             max_concurrent_runs=4)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [7]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values =  best_run.get_details()['runDefinition']['arguments']

print('Best Run Id:', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])

Best Run Id: HD_8d7e12eb-d2a5-44fe-b14b-1fd519901ea9_2

 Accuracy: 0.9139605462822459


In [20]:
#I'm not sure how I can get the model output using hyperdrive, do I save the model in train.py?

2024-03-30:20:24:04,938 INFO     [explanation_client.py:334] Using default datastore for uploads
2024-03-30:20:24:13,826 INFO     [explanation_client.py:334] Using default datastore for uploads


In [9]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

ds = TabularDatasetFactory.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [10]:
from train import clean_data
from sklearn.model_selection import train_test_split


# Use the clean_data function to clean your data.
x, y = clean_data(ds)

#Split data into train and test sets.

x_train,x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [11]:
#we need to feed y column with train column to AutoML, but clean_data separates them
df_train = x_train.copy()
df_train['y'] = y_train

In [12]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=df_train,
    label_column_name='y',
    n_cross_validations=5)

In [13]:
# Submit your automl run

### YOUR CODE HERE ###
remote_run = exp.submit(automl_config)

from azureml.widgets import RunDetails
RunDetails(remote_run).show()

remote_run.wait_for_completion()

2024-03-30:19:28:49,354 WARNING  [local_experiment_driver.py:142] Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_87f708a5-075d-4f25-ae1b-8612c1d589ee,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


2024/03/30 19:42:41 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2024-03-30:20:01:48,489 INFO     [explanation_client.py:334] Using default datastore for uploads


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_87f708a5-075d-4f25-ae1b-8612c1d589ee',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2024-03-30T19:28:52.276955Z',
 'endTimeUtc': '2024-03-30T20:00:46.599752Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 30 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"cdbe0b43-92a0-4715-838a-f2648cc7ad21","resource_group":"aml-quickstarts-256399","workspace_name":"quick-starts-ws-256399","region":"southcentralus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","many_models":false,"pipeline_fetch_max_batch_size":1,

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

2024-03-30:20:20:51,285 INFO     [explanation_client.py:334] Using default datastore for uploads
2024-03-30:20:20:56,825 INFO     [explanation_client.py:334] Using default datastore for uploads


In [14]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: udacity-project,
Id: AutoML_87f708a5-075d-4f25-ae1b-8612c1d589ee_15,
Type: None,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('0', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmclassifier', LightGBMClassifier(min_data_in_leaf=20, n_jobs=1, problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=None))], verbose=False)), ('14', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=Fa

In [19]:
best_run.get_metrics()['accuracy']

0.9175265553869499

In [ ]:
#Accuracy on test set to compare to hyperdrive model

In [15]:
y_pred_test = fitted_model.predict(x_test)
accuracy = fitted_model.score(x_test, y_test)

In [16]:
accuracy

0.917298937784522

In [ ]:
#AutoML 0.917 vs hyperdrive with logistic regression 0.914